In [ ]:
import pandas as pd

from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql.functions import *
from pyspark.sql.types import *
from sklearn.metrics import confusion_matrix

from pyspark.sql.functions import concat, col, lit

from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
data = spark.sql("select mtn,ivrcallid,start_tm_local from vzw_cja_prd_tbls.verint_call_transcripts_raw  \
                    where cast(end_tm_local as timestamp) BETWEEN '2019-11-23 09:00:00' AND '2019-11-24 09:00:00'")

# where day(end_tm_local)=22 AND month(end_tm_local)=11 AND year(end_tm_local)=2019") 

#cast(end_tm_local as timestamp) BETWEEN '2019-11-22 09:00:00' AND '2019-11-13 09:00:00' ORDER BY START_TM_LOCAL")


In [ ]:
driverDF= spark.sql("SELECT ivr_id AS ivrcallid, primary_topic,secondary_topic FROM vzw_soi_prd_tbls.vzsoi_intl_calltopics_v1 where cast(START_TM_LOCAL as timestamp) BETWEEN '2019-11-23 09:00:00' AND '2019-11-24 09:00:00'")
                    
                    #day(START_TM_LOCAL)=22 AND month(START_TM_LOCAL)=11 AND year(START_TM_LOCAL)=2019")


In [ ]:
   joinDF= data.join(driverDF,on=['ivrcallid'],how='inner')\
                 .dropDuplicates(['ivrcallid'])

In [ ]:
joinDF.select("mtn").distinct().show()

In [ ]:
joinDF.count()

In [ ]:
a=joinDF
a.dropDuplicates(['mtn']).count()

In [ ]:
joinDF.limit(10).toPandas()

In [ ]:
split_col = split(joinDF['mtn'], '-')
joinDF = joinDF.withColumn('mtn1', split_col.getItem(0))
joinDF = joinDF.withColumn('mtn2', split_col.getItem(1))
joinDF = joinDF.withColumn('mtn3',split_col.getItem(2))
topictable=joinDF.withColumn("mtnconcat",concat(col("mtn1"), lit(""), col("mtn2"),lit(""),col("mtn3")))


In [ ]:
topictable.limit(10).toPandas()

In [ ]:
predframe= spark.sql("select mtn as mtnconcat,label,score from vzw_soi_prd_tbls.intl_prediction where day(process_dt)=23 AND month(process_dt)=11 AND year(process_dt)=2019")

In [ ]:
predframe.limit(10).toPandas()

In [ ]:
ValidationDF= predframe.join(topictable,on=['mtnconcat'],how='inner')
# joinDF= data.join(driverDF,on=['ivrcallid'],how='inner')\
 #                .dropDuplicates(['ivrcallid'])

In [ ]:
ValidationDF.count()

In [ ]:
ValidationDF.groupby('mtn').toPandas()

In [ ]:
a= ValidationDF.dropDuplicates(['mtn'])
a.count()

In [ ]:
prediction_conf=ValidationDF.select('label','primary_topic')


In [ ]:
(prediction_conf).show()

In [ ]:
prediction_conf = prediction_conf.withColumn('primary_topic', when((col('primary_topic') =='FRAUD')| (col('primary_topic') =='STOLEN & DAMAGED'),'PLAN & TRAVEL PASS').otherwise(col('primary_topic')))
prediction_conf = prediction_conf.withColumn('primary_topic', when(col('primary_topic') =='PHONE SETTINGS','phone_settings').otherwise(col('primary_topic')))
prediction_conf = prediction_conf.withColumn('primary_topic', when(col('primary_topic') =='BILLING','billing').otherwise(col('primary_topic')))
prediction_conf = prediction_conf.withColumn('primary_topic', when(col('primary_topic') =='CRUISE','cruise').otherwise(col('primary_topic')))
prediction_conf = prediction_conf.withColumn('primary_topic', when(col('primary_topic') =='PLAN & TRAVEL PASS','plan_travelpass').otherwise(col('primary_topic')))

In [ ]:
prediction_conf = prediction_conf.withColumn('primary_topic', when(col('primary_topic') =='phone_settings',3.0).otherwise(col('primary_topic')))
prediction_conf = prediction_conf.withColumn('primary_topic', when(col('primary_topic') =='billing',1.0).otherwise(col('primary_topic')))
prediction_conf = prediction_conf.withColumn('primary_topic', when(col('primary_topic') =='cruise',2.0).otherwise(col('primary_topic')))
prediction_conf = prediction_conf.withColumn('primary_topic', when(col('primary_topic') =='plan_travelpass',0.0).otherwise(col('primary_topic')))

In [ ]:
prediction_conf = prediction_conf.withColumn('label', when(col('label') =='phone_settings',3.0).otherwise(col('label')))
prediction_conf = prediction_conf.withColumn('label', when(col('label') =='billing',1.0).otherwise(col('label')))
prediction_conf = prediction_conf.withColumn('label', when(col('label') =='cruise',2.0).otherwise(col('label')))
prediction_conf = prediction_conf.withColumn('label', when(col('label') =='plan_travelpass',0.0).otherwise(col('label')))

In [ ]:
(prediction_conf).show()

In [ ]:
t=prediction_conf.toPandas()

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(t['primary_topic'],t['label'])

In [ ]:
a= spark.sql("SELECT ivr_id AS ivrcallid, primary_topic, secondary_topic,START_TM_LOCAL FROM vzw_soi_prd_tbls.vzsoi_intl_calltopics_v1 WHERE cast(START_TM_LOCAL as timestamp) BETWEEN '2019-11-16 00:00:00' AND '2019-11-22 00:00:00' ORDER BY START_TM_LOCAL")


In [ ]:
a.toPandas()

In [ ]:
a= spark.sql("SELECT count(ivr_id) FROM vzw_soi_prd_tbls.vzsoi_intl_calltopics_v1 WHERE cast(START_TM_LOCAL as timestamp) BETWEEN '2019-11-16 00:00:00' AND '2019-11-18 00:00:00' ORDER BY START_TM_LOCAL")


In [ ]:
--lets do like what surya did, call 1, call 2 , call 3 for same mtn